<a href="https://colab.research.google.com/github/Anthonaut/COMP-390-Tutorial-Image-Classifcation-with-CNN/blob/main/Image_Classification_with_CNN_without_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [ ]:
pip install -q tensorflow tensorflow-datasets

## Import Necessary Libaries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
tfds.disable_progress_bar() # - Gets rid of the progress bars that appear (e.g running "ds_train" and "ds_test")

## Find Datasets

In [ ]:
# https://www.tensorflow.org/datasets/overview
# https://www.tensorflow.org/datasets/catalog/overview
tfds.list_builders()

## Get information on the Data

In [ ]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info

info

## Prepare Rock, Paper, Scissors Data

In [ ]:
ds_train = tfds.load(name="rock_paper_scissors", split="train")
ds_test = tfds.load(name="rock_paper_scissors", split="test")

## Show Examples

In [ ]:
fig = tfds.show_examples(info, ds_train)

## Additional Data Prep

In [ ]:
train_images = np.array([example['image'].numpy()[:,:,0] for example in ds_train])
train_labels = np.array([example['label'].numpy() for example in ds_train])

test_images = np.array([example['image'].numpy()[:,:,0] for example in ds_test])
test_labels = np.array([example['label'].numpy() for example in ds_test])

In [ ]:
# type(train_images[0]) - numpy type
# train_images.shape - (2520, 300, 300), 1st values is # number of images
# test_images.shape - (372, 300, 300)
train_images = train_images.reshape(2520, 300, 300, 1) # Makes images grayscale
test_images = test_images.reshape(372, 300, 300, 1)

# train_images.dtype
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images /= 255
test_images /= 255

## Train a Network (Basic Approach)

In [ ]:
model = keras.Sequential([
  keras.layers.Flatten(),
  keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32)

In [ ]:
model.evaluate(test_images, test_labels)

## Train a Network (Convolutional Approach)

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(64, 3, activation='relu', input_shape=(300,300,1)),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32)

In [ ]:
model.evaluate(test_images, test_labels)

## A Better Convolutional Network - solve the problem of overfitting

In [ ]:
model = keras.Sequential([
  keras.layers.AveragePooling2D(6,3, input_shape=(300,300,1)),
  keras.layers.Conv2D(64, 3, activation='relu'),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Dropout(0.5),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32)

In [ ]:
model.evaluate(test_images, test_labels)

## Hyperparameter Tuning

In [ ]:
pip install keras-tuner --upgrade

In [ ]:
from kerastuner.tuners import RandomSearch

def build_model(hp):
  model = keras.Sequential()

  model.add(keras.layers.AveragePooling2D(6,3, input_shape=(300,300,1)))
  
  # model.add(keras.layers.Conv2D(64, 3, activation='relu'))
  # model.add(keras.layers.Conv2D(32, 3, activation='relu'))
  for i in range(hp.Int("Conv layers", min_value=0, max_value=3)): # Test between 0-3 conv layers, for each layer the number of filters used (i.e [16,32,64])
    model.add(keras.layers.Conv2D(hp.Choice(f"layer_{i}_filters", [16,32,64], 3, activation='relu')))

  model.add(keras.layers.MaxPool2D(2,2))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Flatten())

  model.add(keras.layers.Dense(hp.Choice("Denser layer", [64, 128, 256, 512, 1024]), activation='relu'))

  model.add(keras.layers.Dense(3, activation='softmax'))

  model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
  return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=32
)

tuner.search(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=32)


In [ ]:
best_model = tuner.get_best_models()[0] # Get the best model that has been tested

In [ ]:
best_model.evaluate(test_images, test_labels)

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()

## Save & Load our Models

In [ ]:
best_model.save('./my_model') # Save the model with its parameters

In [ ]:
loaded_model = keras.models.load_model('./my_model') # Use the above two lines of code on your local machine, Colab isn't guaranteed to save it the next time you run it

In [ ]:
loaded_model.evaluate(test_images, test_labels) # Test the saved model

Plot Image from Numpy Array

In [ ]:
image = train_images[0].reshape(300, 300)

plt.imshow(image, cmap='Greys_r')

In [ ]:
 rgb_images=np.array([example['image'].numpy() for example in ds_train.take(1)])
 rgb_image = rgb_images[0]

 plt.imshow(rgb_image)
 rgb_image.shape

## Convert PNG/JPG images to Numpy Format

In [ ]:
import imageio

im = imageio.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/Sacramento%2C-California---State-Capitol_%28cropped%29.jpg/1200px-Sacramento%2C-California---State-Capitol_%28cropped%29.jpg')

plt.imshow(im)

print(type(im))

im_np = np.asarray(im) # Converts to Numpy
print(im_np.shape)